In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import string
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import string

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier

from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.models import load_model


seed = 4353

# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link_fake='https://drive.google.com/open?id=1sTzWwA8JCiBKH9u364c-kezhMnTiHsvI'
fluff, id1 = link_fake.split('=')
print (id1) # Verify that you have everything after '='

In [ ]:
link_true='https://drive.google.com/open?id=1fhYamoD4zvINmJF33sxhaMz7ynpmpsN9'
fluff, id2 = link_true.split('=')
print (id2) # Verify that you have everything after '='

In [ ]:
downloaded = drive.CreateFile({'id':id1}) 
downloaded.GetContentFile('Fake.csv')
downloaded = drive.CreateFile({'id':id2})
downloaded.GetContentFile('True.csv')

In [ ]:
df_fake=pd.read_csv('Fake.csv')
df_true=pd.read_csv('True.csv')

In [ ]:
df_fake.head()

In [ ]:
df_fake.info()

In [ ]:
df_fake.describe(include='all')

In [ ]:
df_fake['subject'].value_counts()

In [ ]:
df_fake['text']=df_fake['title'].astype(str)+df_fake['text'].astype(str)
df_fake['type']=0
df_fake.drop(columns=['title','date'],inplace=True)

In [ ]:
df_true.head()

In [ ]:
df_true['text']=df_true['title'].astype(str)+df_true['text'].astype(str)
df_true['type']=1
df_true.drop(columns=['title','date'],inplace=True)

In [ ]:
df_true['subject'].value_counts()

In [ ]:
df=pd.concat([df_fake,df_true])
df.head()

In [ ]:
df.tail()

In [ ]:
df.describe(include='all')

In [ ]:
df['type'].value_counts()

In [ ]:
df['subject'].value_counts()

In [ ]:
df.info()

In [ ]:
df.iloc[0,0]

In [ ]:
porter = PorterStemmer()
stop_words = stopwords.words('english')

for i in np.arange(0,44898):
    text_tokens = word_tokenize(df.iloc[i,0])
    text_tokens = [w.lower() for w in text_tokens]
    tokens_without_sw = [word for word in text_tokens if not word in stop_words]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens_without_sw]
    words = [word for word in stripped if word.isalpha()]
    stemmed = [porter.stem(word) for word in words]
    join_back = (" ").join(stemmed)
    df.iloc[i,0] = join_back

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df.iloc[0,0]

In [ ]:
df.to_csv('clean.csv')
!cp clean.csv "drive/My Drive/Data Science"

In [ ]:
link_clean='https://drive.google.com/open?id=1pC7daDCEmdINGjTqDMEl60QZucLPc290'
fluff, id3 = link_clean.split('=')
print (id3) # Verify that you have everything after '='

In [ ]:
downloaded = drive.CreateFile({'id':id3}) 
downloaded.GetContentFile('clean.csv')

In [ ]:
df=pd.read_csv('/content/drive/My Drive/Data Science/clean.csv')
df

In [ ]:
# Word extraction from true and fake texts

true_text = df[df.type==1]['text']
true_text = true_text.reset_index().drop(['index'], axis=1)
fake_words = df[df.type==0]['text']
fake_words = fake_words.reset_index().drop(['index'], axis=1)

In [ ]:
def plot_wordcloud(text_d):
    wordcloud = WordCloud(background_color = 'black',
                         max_words = 3000,
                         width=1600,
                         height=800).generate(text_d)
    plt.clf()
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()

In [ ]:
plt.figure(figsize=(20,18))
plot_wordcloud(" ".join(true_text.text))

In [ ]:
plt.figure(figsize=(20,18))
plot_wordcloud(" ".join(fake_words.text))

In [ ]:
# Function to retrieve processed words

def final(X_data_full):
    
    # function for removing punctuations
    def remove_punct(X_data_func):
        string1 = X_data_func.lower()
        translation_table = dict.fromkeys(map(ord, string.punctuation),' ')
        string2 = string1.translate(translation_table)
        return string2
    
    X_data_full_clear_punct = []
    for i in range(len(X_data_full)):
        test_data = remove_punct(X_data_full[i])
        X_data_full_clear_punct.append(test_data)
        
    # function to remove stopwords
    def remove_stopwords(X_data_func):
        pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
        string2 = pattern.sub(' ', X_data_func)
        return string2
    
    X_data_full_clear_stopwords = []
    for i in range(len(X_data_full)):
        test_data = remove_stopwords(X_data_full[i])
        X_data_full_clear_stopwords.append(test_data)
        
    # function for tokenizing
    def tokenize_words(X_data_func):
        words = nltk.word_tokenize(X_data_func)
        return words
    
    X_data_full_tokenized_words = []
    for i in range(len(X_data_full)):
        test_data = tokenize_words(X_data_full[i])
        X_data_full_tokenized_words.append(test_data)
        
    # function for lemmatizing
    lemmatizer = WordNetLemmatizer()
    def lemmatize_words(X_data_func):
        words = lemmatizer.lemmatize(X_data_func)
        return words
    
    X_data_full_lemmatized_words = []
    for i in range(len(X_data_full)):
        test_data = lemmatize_words(X_data_full[i])
        X_data_full_lemmatized_words.append(test_data)
        
    # creating the bag of words model
    cv = CountVectorizer(max_features=1000)
    X_data_full_vector = cv.fit_transform(X_data_full_lemmatized_words).toarray()
    
    
    tfidf = TfidfTransformer()
    X_data_full_tfidf = tfidf.fit_transform(X_data_full_vector).toarray()
    
    return X_data_full_tfidf

In [ ]:
# Preparing training and testing data using train_test_split

# Data preparation

import nltk
nltk.download('wordnet')

X_data = df['text']
y_data = df.type
X_data = X_data.astype(str)

x_data=final(X_data)

NBX_train, NBX_test, NBy_train, NBy_test = train_test_split(x_data, y_data, test_size=0.3, random_state= seed)

In [ ]:
# Instatiation, fitting and prediction

MNB = MultinomialNB()
MNB.fit(NBX_train, NBy_train)
NB_predictions = MNB.predict(NBX_test)

In [ ]:
# Model evaluation

print("Classification Report", "\n" ,classification_report(NBy_test, NB_predictions))
print("Confusion Matrix", "\n" ,confusion_matrix(NBy_test, NB_predictions))

MNB_f1 = round(f1_score(NBy_test, NB_predictions, average='weighted'), 3)
MNB_accuracy = round((accuracy_score(NBy_test, NB_predictions)*100),2)

print("\n","NB Evaluation:","\n","Accuracy : " , MNB_accuracy , " %")
print("f1_score : " , MNB_f1)

In [ ]:
# Instatiation, fitting and prediction

RFX_train, RFX_test, RFy_train, RFy_test = train_test_split(x_data, y_data, test_size=0.3, random_state= seed)

rfc=RandomForestClassifier(n_estimators= 10, random_state= seed)
rfc.fit(RFX_train, RFy_train)
RF_predictions = rfc.predict(RFX_test)

In [ ]:
# Model evaluation

print("Classification Report", "\n" , classification_report(RFy_test, RF_predictions))
print("Confusion Matrix", "\n" ,confusion_matrix(RFy_test, RF_predictions))

rfc_f1 = round(f1_score(RFy_test, RF_predictions, average= 'weighted'), 3)
rfc_accuracy = round((accuracy_score(RFy_test, RF_predictions) * 100), 2)

print("\n","RF Evaluation:","\n","Accuracy : " , rfc_accuracy , " %")
print("f1_score : " , rfc_f1)

In [ ]:
# Function to extract major words from true and fake news

def wordcloud_words(X_data_full):
    
    # function for removing punctuations
    def remove_punct(X_data_func):
        string1 = X_data_func.lower()
        translation_table = dict.fromkeys(map(ord, string.punctuation),' ')
        string2 = string1.translate(translation_table)
        return string2
    
    X_data_full_clear_punct = []
    for i in range(len(X_data_full)):
        test_data = remove_punct(X_data_full[i])
        X_data_full_clear_punct.append(test_data)
        
    # function to remove stopwords
    def remove_stopwords(X_data_func):
        pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
        string2 = pattern.sub(' ', X_data_func)
        return string2
    
    X_data_full_clear_stopwords = []
    for i in range(len(X_data_full)):
        test_data = remove_stopwords(X_data_full[i])
        X_data_full_clear_stopwords.append(test_data)
        
    # function for tokenizing
    def tokenize_words(X_data_func):
        words = nltk.word_tokenize(X_data_func)
        return words
    
    X_data_full_tokenized_words = []
    for i in range(len(X_data_full)):
        test_data = tokenize_words(X_data_full[i])
        X_data_full_tokenized_words.append(test_data)
        
    # function for lemmatizing
    lemmatizer = WordNetLemmatizer()
    def lemmatize_words(X_data_func):
        words = lemmatizer.lemmatize(X_data_func)
        return words
    
    X_data_full_lemmatized_words = []
    for i in range(len(X_data_full)):
        test_data = lemmatize_words(X_data_full[i])
        X_data_full_lemmatized_words.append(test_data)
        
    return X_data_full_lemmatized_words

In [ ]:
fake_words.iloc[50,0]

In [ ]:
#h=["break michael flynn crack testifi mueller trump himselfmichael flynn trump embattl former nation secur advi reportedli cave testifi robert mueller team trump collu russia accord abc news special report flynn plead guilti charg includ make fal statement fbi importantli admit plea offici trump transit team direct contact russian officialsfurthermor accord cnn david wright twitter report brian ross report abc news said flynn also say prepar testifi trump order direct make contact russian contradict donald trump said point brianross report michael flynn prepar testifi presid trump candid donald trump order direct make contact russian contradict donald trump said point david wright davidwrightcnn decemb brianross well told flynn made deci cooper last hour distraught deci feel right thing countri david wright davidwrightcnn decemb brianross face huge legal bill million dollar said final go reason expect put hou market face seriou financ problem david wright davidwrightcnn decemb part flynn issu statement say follow action acknowledg court today wrong faith god work set thing right guilti plea agreement cooper special counsel offic reflect deci made best interest famili countri accept full respon action white hou said mere got flynn fire first place zero effect trump har de har har make us laugh hard hurt merri christma donald trump entir treason famili administr hope like orang jumpsuit featur imag via chip somodevillagetti imag"]

f=[fake_words.iloc[50,0]]
t=[true_text.iloc[50,0]]

h_data=final(t)

rfc.predict(h_data)